In [1]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
# 广州市顺丰100个点的经纬度数据
coordinates = pd.read_excel('广州市中心顺丰100个配送点坐标.xlsx', engine='openpyxl', header=None).to_numpy() # 100个点的经纬度 (lat, lon)

In [2]:
coordinates

array([[ 23.12964 , 113.269317],
       [ 23.123644, 113.267173],
       [ 23.126848, 113.258184],
       [ 23.134941, 113.270174],
       [ 23.123197, 113.256758],
       [ 23.131689, 113.25352 ],
       [ 23.123561, 113.274397],
       [ 23.125465, 113.249594],
       [ 23.120184, 113.251701],
       [ 23.117515, 113.254512],
       [ 23.1259  , 113.282076],
       [ 23.112659, 113.25864 ],
       [ 23.134902, 113.282353],
       [ 23.147034, 113.263756],
       [ 23.144609, 113.274334],
       [ 23.116344, 113.249221],
       [ 23.148544, 113.263592],
       [ 23.141426, 113.281228],
       [ 23.148975, 113.262729],
       [ 23.149256, 113.264311],
       [ 23.120379, 113.284351],
       [ 23.145898, 113.251496],
       [ 23.114886, 113.247453],
       [ 23.139353, 113.243187],
       [ 23.132525, 113.240692],
       [ 23.151288, 113.261162],
       [ 23.148523, 113.251154],
       [ 23.151997, 113.261522],
       [ 23.151492, 113.257057],
       [ 23.152777, 113.263507],
       [ 2

In [3]:
coordinates.shape

(100, 2)

In [4]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
# 广州市顺丰100个点的经纬度数据
coordinates = pd.read_excel('广州市中心顺丰100个配送点坐标.xlsx', engine='openpyxl', header=None).to_numpy() # 100个点的经纬度 (lat, lon)

# 时间相关参数设置
time_steps = 12  # 12个时间步长，每个时间步代表2小时

# 创建100*100*12的矩阵存储访问时间
visit_time_matrix = np.zeros((100, 100, time_steps))

# 定义时间段速度模型（电动车）
average_speeds = {
    0: 50,  # 0-2点
    1: 50,  # 2-4点
    2: 45,  # 4-6点
    3: 40,  # 6-8点
    4: 32,  # 8-10点
    5: 40,  # 10-12点
    6: 55,  # 12-14点
    7: 50,  # 14-16点
    8: 38,  # 16-18点
    9: 30,  # 18-20点
    10: 45, # 20-22点
    11: 50  # 22-24点

}

# 定义基础交通拥堵因子
base_traffic_factors = {
    0: 1.0,   # 夜间，交通通畅
    1: 1.0,   # 夜间，交通通畅
    2: 1.2,   # 黎明，交通开始繁忙
    3: 2.5,   # 早高峰，交通拥堵
    4: 2.0,   # 上午高峰，交通较拥堵
    5: 1.5,   # 中午，交通缓解
    6: 1.3,   # 下午，交通较通畅
    7: 1.8,   # 下午过渡期，交通开始繁忙
    8: 2.0,   # 晚高峰，交通最拥堵
    9: 2.5,   # 晚高峰，交通拥堵
    10: 1.2,  # 夜间，交通恢复正常
    11: 1.0   # 深夜，交通通畅
}

# 定义速度波动因子
volatility = {
    0: 0.05,  # 夜间，交通通畅，波动小
    1: 0.05,  # 夜间，交通通畅，波动小
    2: 0.1,   # 黎明，交通开始繁忙，波动中等
    3: 0.25,  # 早高峰，交通拥堵，波动大
    4: 0.2,   # 上午高峰，交通较拥堵，波动大
    5: 0.15,  # 中午，交通缓解，波动中等
    6: 0.12,  # 下午，交通较通畅，波动中等
    7: 0.18,  # 下午过渡期，交通开始繁忙，波动大
    8: 0.3,   # 晚高峰，交通最拥堵，波动大
    9: 0.28,  # 晚高峰，交通拥堵，波动大
    10: 0.1,  # 夜间，交通恢复正常，波动小
    11: 0.08  # 深夜，交通通畅，波动小
}

# 定义红绿灯和其他城市道路因素的影响因子
traffic_light_factors = {
    0: 1.1,  # 夜间，红绿灯较少
    1: 1.1,  # 夜间，红绿灯较少
    2: 1.2,  # 黎明，红绿灯开始运作
    3: 1.5,  # 早高峰，红绿灯频繁
    4: 1.4,  # 上午高峰，红绿灯较多
    5: 1.3,  # 中午，红绿灯正常
    6: 1.2,  # 下午，红绿灯正常
    7: 1.3,  # 下午过渡期，红绿灯较多
    8: 1.6,  # 晚高峰，红绿灯频繁
    9: 1.7,  # 晚高峰，红绿灯较多
    10: 1.2, # 夜间，红绿灯较少
    11: 1.1  # 深夜，红绿灯较少
}

# 交叉口参数
intersections_per_km = 1  # 每公里交叉口数量
average_wait_time_per_intersection = 3  # 每个交叉口平均等待时间（分钟）

# 计算每对点在每个时间步的访问时间
for i in range(100):
    for j in range(100):
        # 计算两点之间的地理距离
        distance_km = geodesic((coordinates[i, 0], coordinates[i, 1]), 
                             (coordinates[j, 0], coordinates[j, 1])).km

        # 对于每个时间步，动态调整速度和交通因子
        for t in range(time_steps):
            # 计算动态速度
            dynamic_speed = (average_speeds[t] / base_traffic_factors[t]) * (1-np.random.normal(0, volatility[t]))
            dynamic_speed = np.clip(dynamic_speed, 30, 60)  # 限制在30-60 km/h之间

            # 计算基础时间
            travel_time_hours = distance_km / dynamic_speed 

            # 计算红绿灯和其他城市道路因素的影响
            traffic_light_factor = traffic_light_factors[t]
            total_wait_time = intersections_per_km * distance_km * average_wait_time_per_intersection / 60  # 转换为小时

            # 调整后的行程时间
            adjusted_travel_time = travel_time_hours * traffic_light_factor + total_wait_time

            # 存储结果
            visit_time_matrix[i, j, t] = adjusted_travel_time/2

# 保存结果
visit_time_matrix_2d = visit_time_matrix.reshape(-1, visit_time_matrix.shape[-1])
np.savetxt('data.csv', visit_time_matrix_2d, delimiter=',', fmt='%f')

In [5]:
import folium

# 广州市的经纬度
center = [23.129640, 113.269317]  # 广州市的中心坐标

# 广州市100个配送点的经纬度数据 (经度, 纬度)
coordinates = [
    [113.269317, 23.129640], [113.267173, 23.123644], [113.258184, 23.126848],
    [113.270174, 23.134941], [113.256758, 23.123197], [113.253520, 23.131689],
    [113.274397, 23.123561], [113.249594, 23.125465], [113.251701, 23.120184],
    [113.254512, 23.117515], [113.282076, 23.125900], [113.258640, 23.112659],
    [113.282353, 23.134902], [113.263756, 23.147034], [113.274334, 23.144609],
    [113.249221, 23.116344], [113.263592, 23.148544], [113.281228, 23.141426],
    [113.262729, 23.148975], [113.264311, 23.149256], [113.284351, 23.120379],
    [113.251496, 23.145898], [113.247453, 23.114886], [113.243187, 23.139353],
    [113.240692, 23.132525], [113.261162, 23.151288], [113.251154, 23.148523],
    [113.261522, 23.151997], [113.257057, 23.151492], [113.263507, 23.152777],
    [113.285395, 23.115274], [113.260495, 23.153179], [113.290755, 23.134159],
    [113.265293, 23.154109], [113.244610, 23.111918], [113.250732, 23.107016],
    [113.261974, 23.154584], [113.238252, 23.118929], [113.271196, 23.103727],
    [113.291752, 23.136246], [113.278876, 23.106494], [113.252974, 23.153880],
    [113.245845, 23.150375], [113.291045, 23.141334], [113.233769, 23.134541],
    [113.295761, 23.130433], [113.271281, 23.157824], [113.295469, 23.136698],
    [113.237028, 23.144955], [113.250292, 23.102179], [113.281521, 23.103285],
    [113.268110, 23.098888], [113.261307, 23.159572], [113.236696, 23.111871],
    [113.258729, 23.159757], [113.264454, 23.160736], [113.230096, 23.135727],
    [113.259344, 23.161078], [113.297398, 23.140386], [113.292591, 23.150251],
    [113.256043, 23.095810], [113.229532, 23.142626], [113.272076, 23.094993],
    [113.286427, 23.099905], [113.247664, 23.161826], [113.304367, 23.131112],
    [113.303515, 23.120235], [113.241180, 23.097879], [113.223299, 23.122812],
    [113.306526, 23.132445], [113.297364, 23.155081], [113.259261, 23.168992],
    [113.240865, 23.164480], [113.310492, 23.125717], [113.308994, 23.116851],
    [113.262372, 23.085706], [113.310324, 23.139384], [113.248465, 23.170037],
    [113.221709, 23.149001], [113.260136, 23.174135], [113.286623, 23.088690],
    [113.302866, 23.157872], [113.288513, 23.089073], [113.314792, 23.133145],
    [113.309666, 23.149901], [113.221571, 23.153899], [113.215523, 23.115470],
    [113.316224, 23.130867], [113.221810, 23.156805], [113.290182, 23.087220],
    [113.249015, 23.175580], [113.316576, 23.119348], [113.316864, 23.139959],
    [113.262457, 23.179605], [113.311187, 23.155848], [113.296337, 23.087298],
    [113.317861, 23.114620], [113.239624, 23.082741], [113.211440, 23.109301],
    [113.311122, 23.098445],
]

# 创建一个地图对象，设置地图初始位置和缩放级别
m = folium.Map(location=center, zoom_start=12)

# 将所有配送点添加到地图上
for coord in coordinates:
    folium.Marker(location=[coord[1], coord[0]]).add_to(m)  # 坐标顺序为[纬度, 经度]

# 将地图保存为 HTML 文件
m.save("guangzhou_delivery_points.html")